# `System.Object` ‘diff trails’

Libraries like `ObjDiff` [[GitHub](https://github.com/igece/ObjDiff)] produce ‘diff trails’: the value of `Difference<T>.Path` is a _trail_ to the _difference_ between two child objects nested in collections. We can further explain what a ‘diff trail’ is by starting with two simple classes:

In [1]:
public class MyTypeOne
{
    public int Id { get; set; }
    public string Caption { get; set; }
    public ICollection<MyTypeTwo> OneCollection { get; set; } = new List<MyTypeTwo>();
}

public class MyTypeTwo
{
    public int Id { get; set; }
    public string ReferenceName { get; set; }
    public DateTime TimeStamp { get; set; } = DateTime.Now;
    public ICollection<MyTypeTwo> TwoCollection { get; set; } = new List<MyTypeTwo>();
}

## contriving `data`

We can contrive a situation where an object is nested in a collection like our `data` below:

In [2]:
var data = new MyTypeOne
{
    Id = 1,
    Caption = "my data",
    OneCollection = new List<MyTypeTwo>
    {
        new MyTypeTwo
        {
            Id = 10,
            ReferenceName = "L1",
        },
        new MyTypeTwo
        {
            Id = 20,
            ReferenceName = "L1",
            TwoCollection = new List<MyTypeTwo>
            {
                new MyTypeTwo
                {
                    Id = 11,
                    ReferenceName = "L11",
                },
                new MyTypeTwo
                {
                    Id = 12,
                    ReferenceName = "L12",
                },
                new MyTypeTwo
                {
                    Id = 13,
                    ReferenceName = "L13",
                },
                new MyTypeTwo
                {
                    Id = 14,
                    ReferenceName = "L14",
                },
                new MyTypeTwo
                {
                    Id = 15,
                    ReferenceName = "L15",
                    TwoCollection = new List<MyTypeTwo>
                    {
                        new MyTypeTwo
                        {
                            Id = 151,
                            ReferenceName = "L151"
                        }
                    },
                },
            },
        },
    }
};

## a ‘trail’ to our data

We can mimic what might show up in `Difference<T>.Path` (from `ObjDiff` [[GitHub](https://github.com/igece/ObjDiff)]) by setting a variable we call `trail`:

In [3]:
string trail = "OneCollection[1].TwoCollection[4].TwoCollection[0].ReferenceName";

What we want to do is break up this trail into `segments` and turn each segment of this trail into the object it represents in our `data`. We can do this by noticing that a trail segment is delimited by `.`:

In [4]:
string[] segments = trail.Split('.');

segments

[ OneCollection[1], TwoCollection[4], TwoCollection[0], ReferenceName ]

## breaking down a segment representing a collection

The first segment on our trail is `OneCollection[1]` as shown above. This segment has two important bits of information:

1. the collection name in our `data`
2. the index in the collection where our segment object is located

The method,`GetTrailSegmentPair`, should break out these bits into a tuple:

In [5]:
using System.Text.RegularExpressions;

public static (string collectionName, int collectionIndex)? GetTrailSegmentPair(string segment)
{
    if(string.IsNullOrWhiteSpace(segment)) return null;

    if(!segment.EndsWith(']')) return null;

    var match = Regex.Match(segment, @"([^\[\]]+)\[(\d+)\]");

    if (!match.Success)
    {
        return null;
    }

    if (match.Groups.Count != 3)
    {
        return null;
    }

    string collectionName = match.Groups[1].Value;

    if (!int.TryParse(match.Groups[2].Value, out int collectionIndex))
    {
        return null;
    }

    return (collectionName, collectionIndex);
}

## getting the trail objects with a `Queue<string>`

We can use our `GetTrailSegmentPair` with a method we call `GetObjectsFromTrail<TData>` that uses a `Queue<string>` that can be recursively passed after a specified segment is processed and dequeued:

In [6]:
public IReadOnlyCollection<object> GetObjectsFromTrail<TData>(TData data, Queue<string> trailSegments)
{
    string segment = trailSegments.Peek();

    var pair = GetTrailSegmentPair(segment);

    if(pair == null) return Array.Empty<object>();

    trailSegments.Dequeue();

    var (collectionName, collectionIndex) = pair.Value;

    string typeName = typeof(TData).Name;

    List<object> objects = new();

    switch(typeName)
    {
        case nameof(MyTypeOne):
            MyTypeOne mtOne = data as MyTypeOne;
            if(mtOne == null)
            {
                return Array.Empty<object>();
            }

            switch(collectionName)
            {
                case nameof(MyTypeOne.OneCollection):
                    {
                        var target = mtOne.OneCollection.ElementAtOrDefault(collectionIndex);
                        if(target == null)
                        {
                            return Array.Empty<object>();
                        }

                        objects.Add(target);

                        objects.AddRange(GetObjectsFromTrail(target, trailSegments));

                        break;
                    }
            }

            break;

        case nameof(MyTypeTwo):
            MyTypeTwo mtTwo = data as MyTypeTwo;
            if(mtTwo == null)
            {
                return Array.Empty<object>();
            }

            switch(collectionName)
            {
                case nameof(MyTypeTwo.TwoCollection):
                    {
                        var target = mtTwo.TwoCollection.ElementAtOrDefault(collectionIndex);
                        if(target == null)
                        {
                            return Array.Empty<object>();
                        }

                        objects.Add(target);

                        objects.AddRange(GetObjectsFromTrail(target, trailSegments));

                        break;
                    }
            }

            break;
    }

    return objects;
}

Some highlights for `GetObjectsFromTrail<TData>`:

- when `GetTrailSegmentPair` returns `null`, this means the current `segment` does not represent an object in a collection
- `trailSegments` calls `Dequeue` _before_ recursion with a `target` object

## using `GetObjectsFromTrail<TData>`

With `GetObjectsFromTrail<TData>` in place, we can return a collection of objects for every segment on the trail that represents an object in a collection:

In [7]:
var segmentsQueue = new Queue<string>(segments);

GetObjectsFromTrail(data, segmentsQueue)

index value 0 Submission#2+MyTypeTwo Id 20 ReferenceName L1 TimeStamp 2024-10-28 21:21:49Z TwoCollection index value 0 Submission#2+MyTypeTwo Id 11 ReferenceName L11 TimeStamp 2024-10-28 21:21:49Z TwoCollection (empty) 1 Submission#2+MyTypeTwo Id 12 ReferenceName L12 TimeStamp 2024-10-28 21:21:49Z TwoCollection (empty) 2 Submission#2+MyTypeTwo Id 13 ReferenceName L13 TimeStamp 2024-10-28 21:21:49Z TwoCollection (empty) 3 Submission#2+MyTypeTwo Id 14 ReferenceName L14 TimeStamp 2024-10-28 21:21:49Z TwoCollection (empty) 4 Submission#2+MyTypeTwo Id 15 ReferenceName L15 TimeStamp 2024-10-28 21:21:49Z TwoCollection index value 0 Submission#2+MyTypeTwo Id 151 ReferenceName L151 TimeStamp 10/28/2024 9:21:49 PM TwoCollection [ ] 1 Submission#2+MyTypeTwo Id 15 ReferenceName L15 TimeStamp 2024-10-28 21:21:49Z TwoCollection index value 0 Submission#2+MyTypeTwo Id 151 ReferenceName L151 TimeStamp 2024-10-28 21:21:49Z TwoCollection (empty) 2 Submission#2+MyTypeTwo Id 151 ReferenceName L151 TimeStamp 2024-10-28 21:21:49Z TwoCollection (empty)

By reference, we notice there is one element left in the `segmentsQueue` because it does _not_ represent an object in a collection:

In [8]:

segmentsQueue

[ ReferenceName ]

## <!-- -->

[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼